In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2060


In [8]:
import pandas as pd
import numpy as np

n_steps = 5
n_features = 1

def split_sequence(sequence: pd.DataFrame, timestep: int):
    X = list()
    y = list()
    for i in range(len(sequence)):
        end_idx = i + timestep
        if end_idx > len(sequence) - 1:
            break
        seq_x, seq_y = sequence['Close'][i:end_idx].to_numpy(), sequence['Close'][end_idx]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


data = pd.read_csv('../datasets/BRK-A.csv')

X, y = split_sequence(data[:-10000], n_steps)
# X.shape (10000, 5)
# y.shape (10000, 1)

# (batch, n_steps, n_features) => tensorflow(keras) (batch size, number of channels, length of signal sequence.) => pytorch
X = X.reshape(X.shape[0], n_features, X.shape[1])
print(X.shape, y.shape)
print(X[0])
# X.shape (10000, 5, 1)


# ---printout---
# for a, b in zip(X, y):
#     print(a, b)

# print(data['Close'][:10])

(671, 1, 5) (671,)
[[290. 290. 290. 290. 290.]]


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channel=1, out_channel=64, groups=1, bias=True, kernel_size=2, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(2 * 64, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

net = ConvNet()
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, x in enumerate(X, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = x, y

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')